<a href="https://colab.research.google.com/github/IrinaBoyarchukova/A_B_test/blob/main/gb_sem_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [2]:
# data = pd.read_csv('data/gb_sem_9_cw.csv')
data = pd.read_csv('gb_sem_9_cw.csv')

In [3]:
data

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
...,...,...,...,...,...
294473,751197,28:38.6,control,old_page,0
294474,945152,51:57.1,control,old_page,0
294475,734608,45:03.4,control,old_page,0
294476,697314,20:29.0,control,old_page,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         294478 non-null  int64 
 1   time       294478 non-null  object
 2   con_treat  294478 non-null  object
 3   page       294478 non-null  object
 4   converted  294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [5]:
from operator import mul

def convert_time(my_time: str):
    factors = (1, 1/60)
    time = sum(map(mul, map(float, my_time.split(':')), factors))
    return round(time, 2)

data.time = data.time.apply(convert_time)
data.con_treat.replace({'control': 0, 'treatment': 1}, inplace=True)
data.page.replace({'old_page': 0, 'new_page': 1}, inplace=True)

In [6]:
data

,id,time,con_treat,page,converted
0,851104,11.81,0,0,0
1,804228,1.75,0,0,0
2,661590,55.10,1,1,0
3,853541,28.05,1,1,0
4,864975,52.44,0,0,1
...,...,...,...,...,...
294473,751197,28.64,0,0,0
294474,945152,51.95,0,0,0
294475,734608,45.06,0,0,0
294476,697314,20.48,0,0,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         294478 non-null  int64  
 1   time       294478 non-null  float64
 2   con_treat  294478 non-null  int64  
 3   page       294478 non-null  int64  
 4   converted  294478 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 11.2 MB


In [8]:
data.describe()

,id,time,con_treat,page,converted
count,294478.000000,294478.000000,294478.000000,294478.000000,294478.000000
mean,787974.124733,30.052237,0.500126,0.500000,0.119659
std,91210.823776,17.303351,0.500001,0.500001,0.324563
min,630000.000000,0.000000,0.000000,0.000000,0.000000
25%,709032.250000,15.080000,0.000000,0.000000,0.000000
50%,787933.500000,30.060000,1.000000,0.500000,0.000000
75%,866911.750000,45.030000,1.000000,1.000000,0.000000
max,945999.000000,60.000000,1.000000,1.000000,1.000000


In [9]:
data.con_treat.equals(data.page)

False

In [10]:
data.con_treat.compare(data.page).index

Int64Index([    22,    240,    308,    327,    357,    490,    685,    713,
               776,    846,
            ...
            293817, 293888, 293894, 293917, 293996, 294014, 294200, 294252,
            294253, 294331],
           dtype='int64', length=3893)

In [11]:
data.iloc[data.con_treat.compare(data.page).index, :]

,id,time,con_treat,page,converted
22,767017,58.25,0,1,0
240,733976,11.27,0,1,0
308,857184,35.00,1,0,0
327,686623,26.68,1,0,0
357,856078,29.51,1,0,0
...,...,...,...,...,...
294014,813406,25.55,1,0,0
294200,928506,32.17,0,1,0
294252,892498,11.18,1,0,0
294253,886135,49.34,0,1,0


In [12]:
data_2 = data.drop(data.con_treat.compare(data.page).index).copy(deep=True)

In [13]:
data_2.con_treat.equals(data_2.page)

True

In [14]:
data_2.id.value_counts()

773192    2
851104    1
688307    1
718297    1
838144    1
         ..
755610    1
804629    1
837875    1
889019    1
715931    1
Name: id, Length: 290584, dtype: int64

In [15]:
data_2 = data_2.loc[data_2.id.isin(data_2.id.value_counts()[data_2.id.value_counts() == 1].index.values), :]

In [16]:
data_2

,id,time,con_treat,page,converted
0,851104,11.81,0,0,0
1,804228,1.75,0,0,0
2,661590,55.10,1,1,0
3,853541,28.05,1,1,0
4,864975,52.44,0,0,1
...,...,...,...,...,...
294473,751197,28.64,0,0,0
294474,945152,51.95,0,0,0
294475,734608,45.06,0,0,0
294476,697314,20.48,0,0,0


In [17]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [18]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size)
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [19]:
control = data_2[data_2.con_treat == 0].copy(deep=True)
treatment = data_2[data_2.con_treat == 1].copy(deep=True)

In [20]:
### Testing timespent
control.shape, treatment.shape

((145274, 5), (145309, 5))

In [21]:
fig = px.histogram(data,
                   x='time',
                   color = 'con_treat',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

In [22]:
continious_result(control, treatment, 'time')

100%|██████████| 10000/10000 [01:58<00:00, 84.65it/s]


,effect_size,alpha,beta,CI,difference
time,-0.003291,0.367391,0.856139,"[-0.068, 0.184]",0.057918


In [ ]:
### Bucket

In [23]:
for _ in range(100, 1001): 
    if data_2.shape[0] % _ == 0:
        print(_)

249
389
747


In [24]:
n_buckets = 747
data_2 = (data_2
 .sample(n=data_2.shape[0], replace=False)
 .reset_index(drop=True)
 .assign(bucket=list(range(n_buckets)) * int(data_2.shape[0] / n_buckets)))

In [25]:
data_2.head()

,id,time,con_treat,page,converted,bucket
0,739442,3.02,0,0,0,0
1,702805,46.76,1,1,0,1
2,906394,29.36,0,0,0,2
3,867549,1.46,0,0,0,3
4,797970,29.46,0,0,0,4


In [26]:
bucketed_data_2 = data_2.groupby(['con_treat', 'bucket'])['time'].agg(mu=np.mean, std=np.std).reset_index()
bucketed_data_2

,con_treat,bucket,mu,std
0,0,0,29.964925,17.046071
1,0,1,29.981748,17.462139
2,0,2,29.449053,17.376938
3,0,3,29.373061,18.446383
4,0,4,30.677817,18.341967
...,...,...,...,...
1489,1,742,32.249330,17.219222
1490,1,743,31.084308,18.867126
1491,1,744,31.324569,17.158332
1492,1,745,29.463110,17.060143


In [27]:
# Сравним исходное выборочное среднее и среднее бакетных средних 
round(np.mean(data_2["time"]), 5), round(np.mean(bucketed_data_2["mu"]), 5)

(30.05361, 30.05501)

In [28]:
round(np.std(data_2["time"]), 5), round(np.mean(bucketed_data_2["std"]), 5)

(17.30051, 17.28913)

In [29]:
control_bucket = bucketed_data_2[bucketed_data_2.con_treat == 0]
treatment_bucket = bucketed_data_2[bucketed_data_2.con_treat == 1]
continious_result(control_bucket, treatment_bucket, 'mu')

100%|██████████| 10000/10000 [00:02<00:00, 3955.34it/s]


,effect_size,alpha,beta,CI,difference
mu,-0.043858,0.396012,0.864624,"[-0.07, 0.184]",0.05509


In [ ]:
### Testing converted

In [30]:
fig = px.histogram(data_2, x="converted",
                   color='con_treat', barmode='group',
                   height=400)
fig.show()

In [31]:
proportion_result(control, treatment, 'converted')

100%|██████████| 10000/10000 [00:47<00:00, 210.26it/s]


,effect_size,alpha,beta,CI,difference
converted,0.004773,0.192219,0.749144,"[-0.001, 0.004]",0.001589
